In [1]:
import os
import pandas as pd
import numpy as np

tables = {}
table_names = []
for f in os.listdir('./parquet'):
    table_names.append(f.split('.')[0])
    tables[f.split('.')[0]] = pd.read_parquet(f'./parquet/{f}')
table_keys = {i: tables[i].columns.values for i in tables}

In [10]:
import dtale
dtale.show(tables['BIOG_MAIN'], open_browser=True)

/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/dtale/views.py:746: FutureWarning:

['c_tribe', 'c_fl_ey_notes', 'c_fl_ly_notes', 'c_surname', 'c_surname_chn', 'c_mingzi', 'c_mingzi_chn', 'c_notes', 'c_surname_proper', 'c_mingzi_proper', 'c_name_proper', 'c_surname_rm', 'c_mingzi_rm', 'c_name_rm', 'c_created_by', 'c_created_date', 'c_modified_by', 'c_modified_date'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



Exception occurred while processing request: query "tts_sysno=='0.0'" found no data, please alter
Traceback (most recent call last):
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/dtale/views.py", line 111, in _handle_exceptions
    return func(*args, **kwargs)
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/dtale/views.py", line 1866, in test_filter
    run_query(
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/dtale/query.py", line 76, in run_query
    raise Exception('query "{}" found no data, please alter'.format(query))
Exception: query "tts_sysno=='0.0'" found no data, please alter


2022-12-12 18:18:29,560 - ERROR    - Exception occurred while processing request: query "tts_sysno=='0.0'" found no data, please alter
Traceback (most recent call last):
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/dtale/views.py", line 111, in _handle_exceptions
    return func(*args, **kwargs)
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/dtale/views.py", line 1866, in test_filter
    run_query(
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/dtale/query.py", line 76, in run_query
    raise Exception('query "{}" found no data, please alter'.format(query))
Exception: query "tts_sysno=='0.0'" found no data, please alter


In [11]:
tables['BIOG_MAIN'].columns.values

array(['tts_sysno', 'c_personid', 'c_name', 'c_name_chn', 'c_index_year',
       'c_index_year_type_code', 'c_index_year_source_id', 'c_female',
       'c_index_addr_id', 'c_index_addr_type_code', 'c_ethnicity_code',
       'c_household_status_code', 'c_tribe', 'c_birthyear',
       'c_by_nh_code', 'c_by_nh_year', 'c_by_range', 'c_deathyear',
       'c_dy_nh_code', 'c_dy_nh_year', 'c_dy_range', 'c_death_age',
       'c_death_age_range', 'c_fl_earliest_year', 'c_fl_ey_nh_code',
       'c_fl_ey_nh_year', 'c_fl_ey_notes', 'c_fl_latest_year',
       'c_fl_ly_nh_code', 'c_fl_ly_nh_year', 'c_fl_ly_notes', 'c_surname',
       'c_surname_chn', 'c_mingzi', 'c_mingzi_chn', 'c_dy',
       'c_choronym_code', 'c_notes', 'c_by_intercalary',
       'c_dy_intercalary', 'c_by_month', 'c_dy_month', 'c_by_day',
       'c_dy_day', 'c_by_day_gz', 'c_dy_day_gz', 'c_surname_proper',
       'c_mingzi_proper', 'c_name_proper', 'c_surname_rm', 'c_mingzi_rm',
       'c_name_rm', 'c_created_by', 'c_created_date',

In [70]:
person_keys = set()
for i in table_keys:
    if 'tts_sysno' in table_keys[i] or 'c_personid' in table_keys[i]:
        print(i, table_keys[i][0])
        person_keys.update(table_keys[i])
person_keys

ENTRY_DATA tts_sysno
OFFICE_CODES tts_sysno
BIOG_INST_DATA c_personid
POSSESSION_DATA c_personid
STATUS_DATA tts_sysno
EVENTS_DATA c_personid
ALTNAME_DATA tts_sysno
POSTED_TO_ADDR_DATA tts_sysno
TEXT_INSTANCE_DATA tts_sysno
ASSOC_DATA tts_sysno
BIOG_ADDR_DATA tts_sysno
TEXT_CODES tts_sysno
POSTING_DATA c_personid
KIN_DATA tts_sysno
BIOG_TEXT_DATA tts_sysno
POSTED_TO_OFFICE_DATA tts_sysno
TMP_INDEX_YEAR c_personid
EVENTS_ADDR c_event_record_id
POSSESSION_ADDR c_possession_record_id
BIOG_MAIN tts_sysno
BIOG_SOURCE_DATA c_personid


{'c_addr_id',
 'c_addr_type',
 'c_age',
 'c_alt_name',
 'c_alt_name_chn',
 'c_alt_name_type_code',
 'c_appt_type_code',
 'c_assoc_claimer_id',
 'c_assoc_code',
 'c_assoc_count',
 'c_assoc_day',
 'c_assoc_day_gz',
 'c_assoc_id',
 'c_assoc_intercalary',
 'c_assoc_kin_code',
 'c_assoc_kin_id',
 'c_assoc_month',
 'c_assoc_nh_code',
 'c_assoc_nh_year',
 'c_assoc_range',
 'c_assoc_year',
 'c_assume_office_code',
 'c_attempt_count',
 'c_autogen_notes',
 'c_bi_begin_year',
 'c_bi_by_nh_code',
 'c_bi_by_nh_year',
 'c_bi_by_range',
 'c_bi_end_year',
 'c_bi_ey_nh_code',
 'c_bi_ey_nh_year',
 'c_bi_ey_range',
 'c_bi_role_code',
 'c_bibl_cat_code',
 'c_birthyear',
 'c_by_day',
 'c_by_day_gz',
 'c_by_intercalary',
 'c_by_month',
 'c_by_nh_code',
 'c_by_nh_year',
 'c_by_range',
 'c_category_1',
 'c_category_2',
 'c_category_3',
 'c_category_4',
 'c_choronym_code',
 'c_counter',
 'c_created_by',
 'c_created_date',
 'c_day',
 'c_day_ganzhi',
 'c_death_age',
 'c_death_age_range',
 'c_deathyear',
 'c_dele

Executing shutdown due to inactivity...


2022-12-12 19:23:04,318 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2022-12-12 19:23:04,581 - INFO     - Executing shutdown...


Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdown_server
    raise RuntimeError("Not running with the Werkzeug Server

2022-12-12 19:23:04,589 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/dtale/app.py", line 435, in shutdown
    shutdown_server()
  File "/Users/DaweiFu/opt/anaconda3/lib/python3.9/site-packages/dtale/app.py", line 421, in shutdown_server
    raise RuntimeError(

In [23]:
meaning = pd.read_csv('./字段含义.csv')
meaning

,column_code,meaning_cn,meaning_en,data_type,coded,code_table,note
0,c_addr_id,地址ID,address ID,NaN,NaN,NaN,NaN
1,c_addr_cbd,用于数据库维护的临时表,the temporary table for maintenance,NaN,NaN,NaN,NaN
2,c_name,地址名称,address name,NaN,NaN,NaN,NaN
3,c_name_chn,地址中文名称,address Chinese name,NaN,NaN,NaN,NaN
4,c_admin_type,行政区类别,administrative type,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
454,c_text_type_sortorder,书籍分类顺序,the sorted order of text types,NaN,NaN,NaN,NaN
455,c_range,范围,range,NaN,NaN,NaN,NaN
456,c_range_chn,范围中文,range in Chinese,NaN,NaN,NaN,NaN
457,c_approx,时间的相对准确度,relative degree of exactness of a date,NaN,NaN,NaN,NaN


In [72]:
df1 = pd.DataFrame({'字段': [k for k in person_keys if k in meaning['column_code'].values], '中文意思': [meaning['meaning_cn'].values[meaning['column_code']==k][0] for k in person_keys if k in meaning['column_code'].values]})
with open('./人物维度.csv', 'w', encoding='utf-8') as f:
    df1.to_csv(f, index=False, header=True)

In [3]:
neighbor = {}
intermediate = []
for i in table_names:
    if len(table_keys[i]) == 2:
        intermediate.append(i)
    for j in table_names:
        if i==j:
            continue
        if table_keys[i][0] in table_keys[j]:
            if i not in neighbor:
                neighbor[i] = {j: None}
            else:
                neighbor[i].update({j: None})
print(len(neighbor.keys()))
neighbor

77


{'ENTRY_DATA': {'OFFICE_CODES': None,
  'BIOG_INST_DATA': None,
  'STATUS_DATA': None,
  'EVENTS_DATA': None,
  'ALTNAME_DATA': None,
  'POSTED_TO_ADDR_DATA': None,
  'TEXT_INSTANCE_DATA': None,
  'ASSOC_DATA': None,
  'BIOG_ADDR_DATA': None,
  'TEXT_CODES': None,
  'POSTING_DATA': None,
  'KIN_DATA': None,
  'BIOG_TEXT_DATA': None,
  'POSTED_TO_OFFICE_DATA': None,
  'BIOG_MAIN': None},
 'PLACE_CODES': {'ADDR_PLACE_DATA': None},
 'SOCIAL_INSTITUTION_NAME_CODES': {'ENTRY_DATA': None,
  'BIOG_INST_DATA': None,
  'ASSOC_DATA': None,
  'SOCIAL_INSTITUTION_CODES': None,
  'SOCIAL_INSTITUTION_ALTNAME_DATA': None,
  'POSTED_TO_OFFICE_DATA': None,
  'SOCIAL_INSTITUTION_ADDR': None,
  'SOCIAL_INSTITUTION_CODES_CONVERSION': None},
 'ENTRY_TYPES': {'ENTRY_CODE_TYPE_REL': None},
 'OFFICE_CODES': {'ENTRY_DATA': None,
  'BIOG_INST_DATA': None,
  'STATUS_DATA': None,
  'EVENTS_DATA': None,
  'ALTNAME_DATA': None,
  'POSTED_TO_ADDR_DATA': None,
  'TEXT_INSTANCE_DATA': None,
  'ASSOC_DATA': None,
  'BI

In [4]:
for i in neighbor:
    for j in neighbor[i]:
        if table_keys[i][0] == table_keys[j][0]:
            neighbor[i][j] = '一对一'
        else:
            neighbor[i][j] = '一对多'
            

In [5]:
reverse = {}
for i in neighbor:
    for j in neighbor[i]:
        if neighbor[i][j] == '一对多' and j in intermediate:
            if j not in reverse:
                reverse[j] = [i]
            else:
                reverse[j].append(i)

In [6]:
reverse

{'ENTRY_CODE_TYPE_REL': ['ENTRY_TYPES'],
 'CopyMissingTables': ['CopyTables', 'TablesFieldsChanges'],
 'CopyTablesDefault': ['CopyTables', 'TablesFieldsChanges'],
 'ASSOC_CODE_TYPE_REL': ['ASSOC_TYPES'],
 'STATUS_CODE_TYPE_REL': ['STATUS_TYPES'],
 'TEXT_BIBLCAT_CODE_TYPE_REL': ['TEXT_BIBLCAT_TYPES']}

In [10]:
df = pd.DataFrame(columns=['表一', '表二', '表间关系'])
for i in neighbor:
    for j in neighbor[i]:
        df.loc[len(df)] = [i, j, neighbor[i][j]]

In [12]:
for i in reverse:
    if len(reverse[i])==2:
        df.loc[len(df)] = [reverse[i][0], reverse[i][1], '多对多']

In [14]:
with open('./表间关系.csv', 'w', encoding='utf-8') as f:
    df.to_csv(f, index=False)